<a href="https://colab.research.google.com/github/worklifesg/Time-Series-Forecasting-and-Analysis/blob/main/3.%20General%20Forcasting%20Methods/3_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> General Forecasting Methods </h3>

<b> Forecasting Procedure: </b> 

 - Choose Model (through statistical analysis)
 - Split data into train and test sets (for fairly evaluate our model)
 - Fit model on training set
 - Evaluate model on test set
 - Refit model on etire dataset
 - Forecast for future dataset

<b> Section Overview: </b> 

 - Forecasting
 - ACF and PACF plots
 - Autoregression - AR
 - Descriptive Statistics and Tests
 - Choosing ARIMA orders
 - ARIMA based models


---



---



3. ARIMA - AutoRegressive Integrated Moving Average

> Here we have three components
  - AutoRegressive (AR) - generalization of ARMA model
  - Integrated (I)
  - Moving Average (MA)

> a) It works very well with time stamped data but it won't be able to understand any outside factors in forcasting.

> b) Both ARIMA and ARMA are fitted to time series data wither to udnerstand the data or to forecast the data

> <b> Types of ARIMA models </b>
  - Non-Seasonal ARIMA
  - Seasonal ARIMA (SARIMA)
   - Understanding SARIMA with exogeneous variables such as SARIMAX


---

Data --> Non-stationary, where initial differencing step (corresponding to I part of model) can be applied many times to ELIMINATE non-stationary part.

<b> Non-seasonal ARIMA model</b>

> a) Denoted by p,d,q (non-negative integers)
> - p = AR :  A regression model that utilizes dependent relationship between current observation and observation over previous period.
> - d = I: Differencing of observations in order to make time series stationary.
> - d = MA : Model that uses dependency between an observation and residula error from MA applied to lagged observations.

> <b> - Stationary means having constant mean and variance over time. \\
    - It means the future predict will also have constant mean and variance. \\
    - Stationary mathematical test: Augmented Dickey-Fuller Test</b>

> b) Differencing : shifting the data by 1,2.... named as First Difference, Second Difference....each differencing comes with the cost of losing raw data.
  - For seasonald ata we can do differencing by season as well.
  - Another technique is to combine both methods, taking seasonal difference of the first difference.

<b> How to choose p,d,q</b>

  - Method 1 : ACF, PACF plots. Using these plots we can choose p, d, q on viewing the decay n the plot. (Difficult to read the best value)
  - Method 2 : 
    - Grid Search
    - Run ARIMA based models on different combinations of p,d,q and compare models for on some evaluation metric


---

